# **DS 6001: Live Coding 5**
## **Randa Ampah**

In [1]:
import numpy as np
import pandas as pd
import psycopg
from sqlalchemy import create_engine # shortcuts to connect 2 databases in Python
import dotenv
import os
import sqlite3

## **Live Coding**

In [2]:
nba = pd.read_csv('./data/ASA All NBA Raw Data.csv', low_memory=False)

In [3]:
pd.set_option('display.max_rows', 81)
nba.head(3).T

,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
game_date,2022-02-17,2022-02-17,2022-02-17
OT,0,0,0
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5
Team_orb_pct,22.9,22.9,22.9


### Database Normalization
#### First normal form:

1. **All tables must have a primary key**: In this table, `game_id` and `player_id` together are unique on every row, and so they form primary key.
- a primary key is a column or group of columns (called super key when multiple) where each row has a unique value.
2. **All the data must be atomic**: Inactives is non-atomic.
- atomic: one piece of information per cell 
    - can vary from situation
    - ex. do you separate first and last name into different columns? only if you care about both of those values
3. **No repeating groups problem**: We can't solve the non-atomicity problem by creating separate columns if this leads to arbitrary ordering language in the column names (for example, `Inactive1`, `Inactive2`, etc.) and if it leads to a lot of missing data (there would be an `Inactive7` which would be missing any time a team has less than 7 inactive players).

You can't go straight from first normal form to third. You have to go up the ladder. We want to go to third normal form

In [4]:
# We've now achieved first normal form
nba = nba.drop(['Inactives'], axis=1)

In [5]:
#nba.head(3).T # we see that the Inactives column is now gone

### Functional Dependence
Let X and Y be columns in a data table. Y is functionally dependent on X if each value of X has exactly one value of Y.

That's pretty abstract. So here are some guidelines that help me:

1. This use of "function" is the exact same as the concept of a function from algebra and pre-calculus. A correspondence f(x)=y is a function if each value of x has only one associated value of y.

2. X is either a primary key, or something that should be a primary key in another table.

For example, `game_date` (Y) is functionally dependent on `game_id` (X) because one `game_id` takes place on exactly one date.

### Second normal form:
In this table the primary key is a superkey consisting of two columns: `game_id` and `player_id`. 
- for this all of the columns have to depend on everything in the primary key (depends on both player and game id in this case)

2NF is violated if any columns are functionally dependent on part of the primary key but not the entire primary key. This can only happen if the primary key is a superkey.

In [6]:
games = nba[['game_id','game_date','OT','season']].drop_duplicates()
games.head(10)

,game_id,game_date,OT,season
0,202202170BRK,2022-02-17,0,2022
26,202202170CHO,2022-02-17,2,2022
48,202202170LAC,2022-02-17,0,2022
71,202202170MIL,2022-02-17,0,2022
95,202202170NOP,2022-02-17,0,2022
120,202202160BOS,2022-02-16,0,2022
121,202202140WAS,2022-02-14,0,2022
122,202202040DET,2022-02-04,0,2022
123,202202110DET,2022-02-11,0,2022
124,202202100DET,2022-02-10,0,2022


In [7]:
players = nba[['player_id','player']].drop_duplicates()
nba = nba.drop(['player'],axis=1)
players.head(10)

,player_id,player
0,kispeco01,Corey Kispert
1,kuzmaky01,Kyle Kuzma
2,caldwke01,Kentavious Caldwell-Pope
3,netora01,Raul Neto
4,bryanth01,Thomas Bryant
5,avdijde01,Deni Avdija
6,smithis01,Ish Smith
7,hachiru01,Rui Hachimura
8,gillan01,Anthony Gill
9,gaffoda01,Daniel Gafford


In [8]:
nba = nba.drop(['game_date','OT','season'], axis=1)

In [9]:
nba.head(3).T

,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
H_A,A,A,A
Team_Abbrev,WAS,WAS,WAS
Team_Score,117,117,117
Team_pace,94.5,94.5,94.5
Team_efg_pct,0.627,0.627,0.627
Team_tov_pct,13.5,13.5,13.5
Team_orb_pct,22.9,22.9,22.9
Team_ft_rate,0.157,0.157,0.157
Team_off_rtg,123.8,123.8,123.8


### Third normal form:
3NF is violated if there are "transitive dependencies", that is, functional dependence between columns when neither column is part of the primary key.

In [10]:
nba.columns

Index(['game_id', 'H_A', 'Team_Abbrev', 'Team_Score', 'Team_pace',
       'Team_efg_pct', 'Team_tov_pct', 'Team_orb_pct', 'Team_ft_rate',
       'Team_off_rtg', 'Opponent_Abbrev', 'Opponent_Score', 'Opponent_pace',
       'Opponent_efg_pct', 'Opponent_tov_pct', 'Opponent_orb_pct',
       'Opponent_ft_rate', 'Opponent_off_rtg', 'player_id', 'starter', 'mp',
       'fg', 'fga', 'fg_pct', 'fg3', 'fg3a', 'fg3_pct', 'ft', 'fta', 'ft_pct',
       'orb', 'drb', 'trb', 'ast', 'stl', 'blk', 'tov', 'pf', 'pts',
       'plus_minus', 'did_not_play', 'is_inactive', 'ts_pct', 'efg_pct',
       'fg3a_per_fga_pct', 'fta_per_fga_pct', 'orb_pct', 'drb_pct', 'trb_pct',
       'ast_pct', 'stl_pct', 'blk_pct', 'tov_pct', 'usg_pct', 'off_rtg',
       'def_rtg', 'bpm', 'minutes', 'double_double', 'triple_double', 'DKP',
       'FDP', 'SDP', 'DKP_per_minute', 'FDP_per_minute', 'SDP_per_minute',
       'pf_per_minute', 'ts', 'last_60_minutes_per_game_starting',
       'last_60_minutes_per_game_bench', 'PG%', '

In [11]:
team_game = nba[['game_id', 'H_A', 'Team_Abbrev', 'Team_Score', 
                'Team_pace','Team_efg_pct', 'Team_tov_pct', 
                'Team_orb_pct', 'Team_ft_rate',
                'Team_off_rtg', 'Opponent_Abbrev']].drop_duplicates()

In [12]:
player_game = nba.drop(['H_A', 'Team_Score', 
                'Team_pace','Team_efg_pct', 'Team_tov_pct', 
                'Team_orb_pct', 'Team_ft_rate',
                'Team_off_rtg', 'Opponent_Abbrev','Opponent_Score', 
                'Opponent_pace','Opponent_efg_pct', 'Opponent_tov_pct',
                'Opponent_orb_pct','Opponent_ft_rate', 'Opponent_off_rtg'],axis=1)
player_game.head()

,game_id,Team_Abbrev,player_id,starter,mp,fg,fga,fg_pct,fg3,fg3a,...,pf_per_minute,ts,last_60_minutes_per_game_starting,last_60_minutes_per_game_bench,PG%,SG%,SF%,PF%,C%,active_position_minutes
0,202202170BRK,WAS,kispeco01,1,32:30,6,9,0.667,4,6,...,0.061538,9.00,31.716667,22.017778,1.0,36.0,60.0,4.0,0.0,46.253586
1,202202170BRK,WAS,kuzmaky01,1,30:16,2,7,0.286,0,3,...,0.099119,7.44,34.324000,18.475954,0.0,0.0,4.0,85.0,11.0,52.152590
2,202202170BRK,WAS,caldwke01,1,25:26,3,7,0.429,1,3,...,0.000000,7.00,29.820290,16.051693,0.0,32.0,67.0,0.0,0.0,47.021807
3,202202170BRK,WAS,netora01,1,20:40,5,7,0.714,1,1,...,0.048387,7.88,29.920833,14.603922,90.0,10.0,0.0,0.0,0.0,27.603314
4,202202170BRK,WAS,bryanth01,1,14:04,5,6,0.833,0,1,...,0.000000,6.88,20.095833,14.538095,0.0,0.0,0.0,0.0,100.0,36.472537


## **Our Four Tables**

    1. player_game

In [13]:
player_game.head(3).T

,0,1,2
game_id,202202170BRK,202202170BRK,202202170BRK
Team_Abbrev,WAS,WAS,WAS
player_id,kispeco01,kuzmaky01,caldwke01
starter,1,1,1
mp,32:30,30:16,25:26
fg,6,2,3
fga,9,7,7
fg_pct,0.667,0.286,0.429
fg3,4,0,1
fg3a,6,3,3


    2. team_game

In [14]:
team_game.head(3).T

,0,13,26
game_id,202202170BRK,202202170BRK,202202170CHO
H_A,A,H,A
Team_Abbrev,WAS,BRK,MIA
Team_Score,117,103,111
Team_pace,94.5,94.5,88.8
Team_efg_pct,0.627,0.483,0.471
Team_tov_pct,13.5,13.1,11.1
Team_orb_pct,22.9,33.3,26.8
Team_ft_rate,0.157,0.191,0.147
Team_off_rtg,123.8,109.0,103.4


    3. players

In [ ]:
players # could include other info (ex. college, when drafted, etc.)

,player_id,player
0,kispeco01,Corey Kispert
1,kuzmaky01,Kyle Kuzma
2,caldwke01,Kentavious Caldwell-Pope
3,netora01,Raul Neto
4,bryanth01,Thomas Bryant
...,...,...
109702,frazimi01,Michael Frazier
110441,howarwi01,William Howard
110913,mbahalu01,Luc Mbah a Moute
111399,bowmaky01,Ky Bowman


    4. games

In [17]:
games

,game_id,game_date,OT,season
0,202202170BRK,2022-02-17,0,2022
26,202202170CHO,2022-02-17,2,2022
48,202202170LAC,2022-02-17,0,2022
71,202202170MIL,2022-02-17,0,2022
95,202202170NOP,2022-02-17,0,2022
...,...,...,...,...
108259,202001080GSW,2020-01-08,0,2020
108887,202008020HOU,2020-08-02,0,2020
109683,201911060HOU,2019-11-06,0,2020
110125,201912250GSW,2019-12-25,0,2020


## **Documenting Databases with Python**

In [18]:
def pandas_df_to_dbml(df: pd.DataFrame, table_name: str) -> str:
    """
    Converts a pandas DataFrame to a DBML string.

    Args:
        df: The pandas DataFrame to convert.
        table_name: The name of the table in the DBML schema.

    Returns:
        A DBML string representing the DataFrame schema.
    """

    dbml_string = f"Table {table_name} {{\n"

    for column_name, column_type in df.dtypes.items():
        dbml_type = map_pandas_dtype_to_dbml_type(column_type)
        dbml_string += f"  {column_name} {dbml_type}\n"

    dbml_string += "}\n"
    return dbml_string

def map_pandas_dtype_to_dbml_type(dtype) -> str:
    """Maps a pandas dtype to a DBML type."""
    dtype_name = str(dtype)
    if "int" in dtype_name:
      return "int"
    if "float" in dtype_name:
      return "float"
    if "datetime" in dtype_name:
        return "datetime"
    return "varchar"

In [23]:
print(pandas_df_to_dbml(player_game,'player_game'))

Table player_game {
  game_id varchar
  Team_Abbrev varchar
  player_id varchar
  starter int
  mp varchar
  fg int
  fga int
  fg_pct float
  fg3 int
  fg3a int
  fg3_pct float
  ft int
  fta int
  ft_pct float
  orb int
  drb int
  trb int
  ast int
  stl int
  blk int
  tov int
  pf int
  pts int
  plus_minus int
  did_not_play int
  is_inactive int
  ts_pct float
  efg_pct float
  fg3a_per_fga_pct float
  fta_per_fga_pct float
  orb_pct float
  drb_pct float
  trb_pct float
  ast_pct float
  stl_pct float
  blk_pct float
  tov_pct float
  usg_pct float
  off_rtg int
  def_rtg int
  bpm float
  minutes float
  double_double int
  triple_double int
  DKP float
  FDP float
  SDP float
  DKP_per_minute varchar
  FDP_per_minute varchar
  SDP_per_minute varchar
  pf_per_minute float
  ts float
  last_60_minutes_per_game_starting float
  last_60_minutes_per_game_bench float
  PG% float
  SG% float
  SF% float
  PF% float
  C% float
  active_position_minutes float
}



In [24]:
print(pandas_df_to_dbml(team_game,'team_game'))

Table team_game {
  game_id varchar
  H_A varchar
  Team_Abbrev varchar
  Team_Score int
  Team_pace float
  Team_efg_pct float
  Team_tov_pct float
  Team_orb_pct float
  Team_ft_rate float
  Team_off_rtg float
  Opponent_Abbrev varchar
}



In [25]:
print(pandas_df_to_dbml(players,'players'))

Table players {
  player_id varchar
  player varchar
}



In [ ]:
print(pandas_df_to_dbml(games,'games'))

Table games {
  game_id varchar
  game_date varchar
  OT int
  season int
}



Link to DB Docs Table: https://dbdocs.io/ampahranda03/NBA-DS6001-DB-Practice

In [ ]:
myquery = '''
SELECT *
FROM team_game t
INNER JOIN team_game s
ON
'''

pd.read_sql_query(myquery,con=engine)

1st Normal Form
    
1. every table has a primary key
2. nothing non-atomic
3. break up non-atomic data in different rows, not columns

2nd Normal Form
   
1. already in 1NF
2. all columns outside of the primary key columns depend on ALL the primary key columns, not just some of them

3rd Normal Form

1. already in 2NF
2. for all columns outside of the primary key, nothing depends on another column outside the primary key